In [1]:
# Import AutoRec model and data utilities
from utils.model import AutoRec
from utils.autorecdata import AutoRecData
from utils.preprocessor import PreProcessor
# Import external libraries
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm

print("=" * 70)
print("Imports Successful")
print("=" * 70)

Imports Successful


/Users/abbas/Documents/Codes/thesis/NCF/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), '..', 'data', 'ml-1m')
data_path = os.path.join(data_dir, 'ratings.dat')

# Check if file exists
if not os.path.exists(data_path):
    raise FileNotFoundError(
        f"Data file not found at: {data_path}\n"
        f"Please ensure the MovieLens 1M dataset is downloaded and extracted."
    )

def load_ml_1m_data(data_path = data_path) -> pd.DataFrame:  
    print("=" * 70)
    print("Loading MovieLens 1M Dataset")
    print("=" * 70)
    print(f"Data path: {data_path}")
    return pd.read_csv(
        data_path,
        sep='::',
        header=None,
        names=['user_id', 'item_id', 'rating', 'timestamp'],
        engine='python',  # Explicitly use python engine to avoid warning
        dtype={
            'user_id': np.int32,
            'item_id': np.int32,
            'rating': np.float32,
            'timestamp': np.int32
        }
    )


# Load ratings data with proper engine to avoid warnings
print("\nLoading ratings data...")

ratings_df = load_ml_1m_data()

print(f"✓ Successfully loaded {len(ratings_df):,} ratings")


Loading ratings data...
Loading MovieLens 1M Dataset
Data path: /Users/abbas/Documents/Codes/thesis/NCF/src/../data/ml-1m/ratings.dat
✓ Successfully loaded 1,000,209 ratings


In [3]:
# Create PreProcessor instance
preprocessor = PreProcessor()

# Preprocess the data: split into train/test and create rating matrices
print("\nSplitting data into train/test sets and creating rating matrices...")
print("=" * 70)
train_mat, test_mat, num_users, num_items = preprocessor.preprocess_ml1m_data(
    ratings_df, 
    test_size=0.2,  # 20% for testing
    random_state=42  # For reproducibility
)

print(f"✓ Data preprocessing complete!")
print("=" * 70)


Splitting data into train/test sets and creating rating matrices...
✓ Data preprocessing complete!


In [4]:
train_set = AutoRecData(data=train_mat)
test_set = AutoRecData(data=test_mat)

In [5]:
train_loader = data.DataLoader(
    dataset=train_set,
    batch_size=32,
    shuffle=True,
    num_workers=4,
)
print('Train loader created')
print("=" * 70)
test_loader = data.DataLoader(
    dataset=test_set, batch_size=len(test_set), shuffle=False, num_workers=0
)
print('Test loader created')
print("=" * 70)

Train loader created
Test loader created


In [6]:
# Set device (CPU or CUDA if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = AutoRec(
    num_users=num_users,
    num_items=num_items,
    num_hidden_units=500,
).to(device)

Using device: cpu


In [ ]:
loss_f = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=0.001,
    weight_decay=0,
)

best_epoch, best_rmse = 0, np.inf
best_hr_at_10, best_ndcg_at_10 = 0.0, 0.0


print(model)

AutoRec(
  (encoder): Sequential(
    (0): Linear(in_features=3706, out_features=500, bias=True)
    (1): Sigmoid()
  )
  (decoder): Sequential(
    (0): Linear(in_features=500, out_features=3706, bias=True)
  )
)


In [ ]:
from utils.helper import get_metrics, get_ranking_metrics
for epoch in tqdm(range(40)):
    model.train()
    for input_vec in train_loader:
        input_mask = (input_vec > 0).to(device)
        input_vec = input_vec.float().to(device)

        model.zero_grad()
        reconstruction = model(input_vec)
        loss = loss_f(reconstruction * input_mask, input_vec * input_mask)
        loss.backward()
        optimizer.step()

    model.eval()
    rmse = get_metrics(model=model, train_set=train_set, test_set=test_set, device=device)
    hr_at_10, ndcg_at_10 = get_ranking_metrics(
        model=model, 
        train_set=train_set, 
        test_set=test_set, 
        top_k=10, 
        device=device
    )

    print(f"[Epoch {epoch}]:: RMSE: {rmse:.6f}, HR@10: {hr_at_10:.6f}, NDCG@10: {ndcg_at_10:.6f}")

    print("=" * 70)

    if rmse < best_rmse:
        best_rmse, best_epoch = rmse, epoch
        best_hr_at_10, best_ndcg_at_10 = hr_at_10, ndcg_at_10
        # Save best model
        model_dir = os.path.join(os.path.dirname(os.getcwd()), '..', 'models')
        os.makedirs(model_dir, exist_ok=True)
        best_model_path = os.path.join(model_dir, 'AutoRec-best.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'rmse': rmse,
            'hr_at_10': hr_at_10,
            'ndcg_at_10': ndcg_at_10,
            'num_users': num_users,
            'num_items': num_items,
            'num_hidden_units': 500,
        }, best_model_path)

    print(f"Done. Best epoch {best_epoch}, best_rmse: {best_rmse:.6f}.")
    print(epoch)
    print("=" * 70)

# Save final model after training
print("\n" + "=" * 70)
print("Saving final model...")
print("=" * 70)
model_dir = os.path.join(os.path.dirname(os.getcwd()), '..', 'models')
os.makedirs(model_dir, exist_ok=True)
final_model_path = os.path.join(model_dir, 'AutoRec.pth')
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'rmse': rmse,
    'hr_at_10': hr_at_10,
    'ndcg_at_10': ndcg_at_10,
    'num_users': num_users,
    'num_items': num_items,
    'num_hidden_units': 500,
    'best_epoch': best_epoch,
    'best_rmse': best_rmse,
    'best_hr_at_10': best_hr_at_10,
    'best_ndcg_at_10': best_ndcg_at_10,
}, final_model_path)
print(f"✓ Final model saved to: {final_model_path}")
print(f"✓ Best model (epoch {best_epoch}) saved to: {os.path.join(model_dir, 'AutoRec-best.pth')}")
print(f"\nBest metrics:")
print(f"  RMSE: {best_rmse:.6f}")
print(f"  HR@10: {best_hr_at_10:.6f}")
print(f"  NDCG@10: {best_ndcg_at_10:.6f}")
print("=" * 70)

  2%|▎         | 1/40 [00:31<20:30, 31.54s/it]

[Epoch 0]:: RMSE: 1.096847, HR@10: 0.001987, NDCG@10: 0.000153
Done. Best epoch 0 best_rmse: 1.096847.
0


  5%|▌         | 2/40 [01:01<19:26, 30.71s/it]

[Epoch 1]:: RMSE: 1.058839, HR@10: 0.006128, NDCG@10: 0.000503
Done. Best epoch 1 best_rmse: 1.058839.
1


  8%|▊         | 3/40 [01:32<19:02, 30.89s/it]

...

[Epoch 37]:: RMSE: 0.984201, HR@10: 0.145412, NDCG@10: 0.018827
Done. Best epoch 37 best_rmse: 0.953895.
37


 98%|█████████▊| 39/40 [20:41<00:32, 32.28s/it]

[Epoch 38]:: RMSE: 0.983071, HR@10: 0.138125, NDCG@10: 0.017024
Done. Best epoch 38 best_rmse: 0.953895.
38


100%|██████████| 40/40 [21:13<00:00, 31.84s/it]

[Epoch 39]:: RMSE: 0.984097, HR@10: 0.116429, NDCG@10: 0.013393
Done. Best epoch 39 best_rmse: 0.953895.
39


In [ ]:
# Example: Load saved model for inference
# Uncomment and run this cell to load a saved model

# from utils.model import AutoRec
# import torch
# 
# # Load checkpoint
# model_dir = os.path.join(os.path.dirname(os.getcwd()), '..', 'models')
# checkpoint_path = os.path.join(model_dir, 'AutoRec.pth')  # or 'AutoRec-best.pth' for best model
# 
# checkpoint = torch.load(checkpoint_path, map_location='cpu')
# 
# # Recreate model with saved parameters
# loaded_model = AutoRec(
#     num_users=checkpoint['num_users'],
#     num_items=checkpoint['num_items'],
#     num_hidden_units=checkpoint['num_hidden_units']
# )
# 
# # Load model weights
# loaded_model.load_state_dict(checkpoint['model_state_dict'])
# loaded_model.eval()
# 
# print(f"Model loaded from: {checkpoint_path}")
# print(f"Epoch: {checkpoint['epoch']}")
# print(f"RMSE: {checkpoint['rmse']:.6f}")
# print(f"HR@10: {checkpoint['hr_at_10']:.6f}")
# print(f"NDCG@10: {checkpoint['ndcg_at_10']:.6f}")
# 
# # Now you can use loaded_model for inference
